## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
path = "/Users/ecem/Desktop/gyrocardiogram/multiclass_data/"
diseased = np.load(path + "data/diseased-10sec.npy", allow_pickle= True)
print(diseased.shape)

In [4]:
diseased = np.load(path + "data/diseased-10sec.npy", allow_pickle= True)


In [5]:
print(diseased.shape)


(3, 2106, 2560)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [7]:
def zero_crossing(array):
    zero_crossing_rate = np.ones(shape = (array.shape[0],array.shape[1], 7))
    
    for i in range(array.shape[1]):
        zero_crossing_rate[0][i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[0][i]))
        zero_crossing_rate[1][i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[1][i]))
        zero_crossing_rate[2][i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[2][i]))

        zero_crossing_rate[0][i][6] = sum(lbs.zero_crossings(np.array(array[0][i])))
        zero_crossing_rate[1][i][6] = sum(lbs.zero_crossings(np.array(array[1][i])))
        zero_crossing_rate[2][i][6] = sum(lbs.zero_crossings(np.array(array[2][i])))
        
    return zero_crossing_rate

        

In [8]:
zcr_diseased = zero_crossing(diseased)

In [9]:
print(zcr_diseased.shape)

(3, 2106, 7)


In [10]:
zcr_diseased[0][6][4]

0.0908203125

In [11]:
columns_zcr = []
for i in range(zcr_diseased.shape[2]-1):
    columns_zcr.append("zcr_x{}".format(i))
    columns_zcr.append("zcr_y{}".format(i))
    columns_zcr.append("zcr_z{}".format(i))
    
columns_zcr.append("#ofcrossing_x")
columns_zcr.append("#ofcrossing_y")
columns_zcr.append("#ofcrossing_z")

print(columns_zcr)

['zcr_x0', 'zcr_y0', 'zcr_z0', 'zcr_x1', 'zcr_y1', 'zcr_z1', 'zcr_x2', 'zcr_y2', 'zcr_z2', 'zcr_x3', 'zcr_y3', 'zcr_z3', 'zcr_x4', 'zcr_y4', 'zcr_z4', 'zcr_x5', 'zcr_y5', 'zcr_z5', '#ofcrossing_x', '#ofcrossing_y', '#ofcrossing_z']


In [12]:
zcr_diseased[0][i][0:6].tolist()

[0.0654296875, 0.095703125, 0.12255859375, 0.1318359375, 0.0927734375, 0.0625]

In [13]:
def dataframe_creator(array, df = pd.DataFrame(), l = 7, columns = None):
    values = []

    for i in range(array.shape[1]):
        llist = list(array[0][i][0:l-1].tolist() + array[1][i][0:l-1].tolist() 
                        + array [2][i][0:l-1].tolist())
        llist.append(array[0][i][l-1])
        llist.append(array[1][i][l-1])
        llist.append(array[2][i][l-1])

        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [14]:
df = dataframe_creator(zcr_diseased, columns = columns_zcr)

df.reset_index(drop = True, inplace = True)
df

,zcr_x0,zcr_y0,zcr_z0,zcr_x1,zcr_y1,zcr_z1,zcr_x2,zcr_y2,zcr_z2,zcr_x3,...,zcr_z3,zcr_x4,zcr_y4,zcr_z4,zcr_x5,zcr_y5,zcr_z5,#ofcrossing_x,#ofcrossing_y,#ofcrossing_z
0,0.059570,0.090820,0.118164,0.130859,0.093750,0.062500,0.088867,0.152832,0.210449,0.246094,...,0.121094,0.065430,0.093750,0.123047,0.121094,0.084961,0.056641,315.0,563.0,309.0
1,0.072266,0.119141,0.153320,0.156250,0.119629,0.073242,0.106445,0.151367,0.199707,0.187988,...,0.091309,0.040039,0.065430,0.091797,0.100586,0.079102,0.053711,395.0,498.0,245.0
2,0.067383,0.112305,0.146484,0.152344,0.111816,0.067383,0.111328,0.171875,0.217285,0.214355,...,0.097168,0.056641,0.086914,0.114258,0.115723,0.088379,0.058105,369.0,552.0,298.0
3,0.073242,0.103516,0.133789,0.114746,0.083496,0.053223,0.093750,0.145020,0.204590,0.207031,...,0.110352,0.055664,0.091797,0.116211,0.112305,0.085938,0.049316,322.0,525.0,291.0
4,0.058105,0.092285,0.126465,0.126953,0.093750,0.059570,0.109375,0.157227,0.197754,0.202148,...,0.098145,0.063477,0.094727,0.115234,0.117676,0.083008,0.051758,312.0,524.0,301.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,0.032715,0.050293,0.071289,0.074707,0.058105,0.040527,0.049316,0.071777,0.097168,0.095215,...,0.047363,0.077637,0.122559,0.157715,0.158203,0.118164,0.073242,187.0,245.0,402.0
2102,0.030762,0.046387,0.060059,0.061523,0.047363,0.031738,0.039551,0.059082,0.074707,0.076660,...,0.040039,0.085449,0.129883,0.173340,0.166504,0.124512,0.079590,161.0,204.0,431.0
2103,0.030273,0.048340,0.066895,0.069824,0.054688,0.036621,0.041504,0.056641,0.080078,0.086914,...,0.051758,0.089355,0.131348,0.178711,0.186035,0.141113,0.099121,175.0,223.0,473.0
2104,0.033203,0.045898,0.062500,0.058105,0.044922,0.032227,0.041504,0.060059,0.078613,0.080566,...,0.038086,0.080078,0.117188,0.159668,0.165527,0.130859,0.093750,161.0,202.0,434.0


In [15]:
df.to_csv(path + 'feature_extr/four-disease-zcrs.csv')

### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
import librosa.display

In [18]:
lbs.feature.chroma_stft(np.array(diseased[0][i]), sr= 256, hop_length = 512)[1].mean()

0.31897986953792873

In [19]:
def chromagram_average(array):
    pitch = np.ones(shape = (array.shape[0],array.shape[1], 12)) 
    
    for i in range(array.shape[1]):
        for j in range(12):
            pitch[0][i][j] = lbs.feature.chroma_stft(np.array(array[0][i]), sr= 256, hop_length = 512)[j].mean()
            pitch[1][i][j] = lbs.feature.chroma_stft(np.array(array[1][i]), sr= 256, hop_length = 512)[j].mean()
            pitch[2][i][j] = lbs.feature.chroma_stft(np.array(array[2][i]), sr= 256, hop_length = 512)[j].mean()
        
    return pitch

In [20]:
chro_diseased = chromagram_average(diseased)

In [21]:
print(chro_diseased.shape)

(3, 2106, 12)


In [22]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [23]:
col_chro = []
for i in range(chro_diseased.shape[2]):
    col_chro.append(lst[i]+ "_x")
    col_chro.append(lst[i]+ "_y")
    col_chro.append(lst[i]+ "_z")
    
print(col_chro)
print(len(col_chro))

['C_x', 'C_y', 'C_z', 'C♯_x', 'C♯_y', 'C♯_z', 'D_x', 'D_y', 'D_z', 'D♯_x', 'D♯_y', 'D♯_z', 'E_x', 'E_y', 'E_z', 'F_x', 'F_y', 'F_z', 'F♯_x', 'F♯_y', 'F♯_z', 'G_x', 'G_y', 'G_z', 'G♯_x', 'G♯_y', 'G♯_z', 'A_x', 'A_y', 'A_z', 'A♯_x', 'A♯_y', 'A♯_z', 'B_x', 'B_y', 'B_z']
36


In [24]:
ch_df = dataframe_creator(chro_diseased,
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C_x,C_y,C_z,C♯_x,C♯_y,C♯_z,D_x,D_y,D_z,D♯_x,...,G♯_z,A_x,A_y,A_z,A♯_x,A♯_y,A♯_z,B_x,B_y,B_z
0,0.506433,0.370125,0.323123,0.383641,0.41879,0.608553,0.552004,0.672396,0.795379,0.844438,...,0.540427,0.532482,0.537383,0.639506,0.701467,0.866884,0.931928,0.955682,0.9096,0.976065
1,0.499741,0.344504,0.327186,0.399169,0.456624,0.578449,0.549838,0.626352,0.806538,0.850074,...,0.513613,0.521124,0.54084,0.604759,0.727653,0.779073,0.837661,0.978571,0.966666,1.0
2,0.462086,0.370492,0.351161,0.391218,0.417949,0.626253,0.707392,0.733124,0.817953,0.911104,...,0.49622,0.518376,0.591903,0.619725,0.712624,0.826865,0.915158,0.925569,0.962287,1.0
3,0.507259,0.333335,0.369582,0.383199,0.495251,0.588087,0.695756,0.679404,0.718916,0.858398,...,0.454881,0.554078,0.653378,0.691066,0.669254,0.914142,0.964483,0.986184,0.959988,0.938678
4,0.496486,0.309573,0.39156,0.401512,0.446368,0.534617,0.724115,0.646803,0.797483,0.890395,...,0.391695,0.43192,0.520678,0.548201,0.5654,0.682132,0.851218,0.889,1.0,0.958231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,0.487247,0.380534,0.553979,0.64776,0.816192,0.991514,0.6801,0.608633,0.492849,0.757673,...,0.909993,0.658318,0.516163,0.579417,0.4886,0.80132,0.619739,0.932921,0.88882,0.946635
2102,0.521929,0.419225,0.486905,0.651833,0.787943,0.97287,0.712312,0.664055,0.488902,0.745559,...,0.873057,0.674199,0.629744,0.656249,0.536155,0.872828,0.755182,0.942336,0.814954,0.986435
2103,0.511541,0.45333,0.614353,0.633771,0.772193,0.985077,0.708226,0.721785,0.534118,0.866953,...,0.976158,0.729124,0.652433,0.702468,0.500179,0.712968,0.642459,0.861212,0.8748,0.843556
2104,0.545974,0.456226,0.574609,0.693732,0.862159,0.988987,0.674248,0.677486,0.55192,0.772465,...,0.949705,0.74859,0.643407,0.611997,0.502888,0.8207,0.701178,0.898014,0.799819,0.896401


In [25]:
ch_df.to_csv(path + "feature_extr/four-disease-chromogram.csv")

## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [26]:
def tempo_average(array): # 384, 11
    means= []
    for i in range(array.shape[0]): #384
        means.append(array[i][0:11].mean())
    return np.array(means) # 384
        

In [27]:
def tempogram(array = 0, hop_lenght = 256):
    hop_length = 256
    sr =256
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sr, 
                                            hop_length=hop_length)

    times = librosa.times_like(oenv, sr=sr, hop_length=hop_length)

    tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr= sr,
                                              hop_length=hop_length)
    # Estimate the global tempo for display purposes
    tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sr,
                                   hop_length=hop_length)[0]
    return [tempogram, tempo]

In [28]:
tempogram(diseased[0][0])[0].shape

(384, 11)

In [29]:
tempogram(diseased[0][0])[0][12]

array([0.44815585, 0.4504367 , 0.45272643, 0.45502522, 0.45733324,
       0.45965069, 0.46197774, 0.46431458, 0.46666139, 0.46901835,
       0.47138564])

In [30]:
diseased[0][0].shape

(2560,)

In [31]:
def temp_values(array):
    final = np.ones(shape = (3, array.shape[1], 384))
    for i in range(array.shape[1]): #143
        
        _x = tempogram(array[0][i])[0] #1,2560
        _y = tempogram(array[1][i])[0]
        _z = tempogram(array[2][i])[0]

        final[0][i] = tempo_average(_x) #384
        final[1][i] = tempo_average(_y)
        final[2][i] = tempo_average(_z)
        
        
    return final

In [32]:
diseased_ = temp_values(diseased)


In [33]:
diseased_.shape

(3, 2106, 384)

In [34]:
print(diseased_.shape)

(3, 2106, 384)


In [35]:
columns_t = []
for i in range(diseased_.shape[-1]):
    columns_t.append("tempogram_x{}".format(i))
    columns_t.append("tempogram_y{}".format(i))
    columns_t.append("tempogram_z{}".format(i))
print(columns_t)

['tempogram_x0', 'tempogram_y0', 'tempogram_z0', 'tempogram_x1', 'tempogram_y1', 'tempogram_z1', 'tempogram_x2', 'tempogram_y2', 'tempogram_z2', 'tempogram_x3', 'tempogram_y3', 'tempogram_z3', 'tempogram_x4', 'tempogram_y4', 'tempogram_z4', 'tempogram_x5', 'tempogram_y5', 'tempogram_z5', 'tempogram_x6', 'tempogram_y6', 'tempogram_z6', 'tempogram_x7', 'tempogram_y7', 'tempogram_z7', 'tempogram_x8', 'tempogram_y8', 'tempogram_z8', 'tempogram_x9', 'tempogram_y9', 'tempogram_z9', 'tempogram_x10', 'tempogram_y10', 'tempogram_z10', 'tempogram_x11', 'tempogram_y11', 'tempogram_z11', 'tempogram_x12', 'tempogram_y12', 'tempogram_z12', 'tempogram_x13', 'tempogram_y13', 'tempogram_z13', 'tempogram_x14', 'tempogram_y14', 'tempogram_z14', 'tempogram_x15', 'tempogram_y15', 'tempogram_z15', 'tempogram_x16', 'tempogram_y16', 'tempogram_z16', 'tempogram_x17', 'tempogram_y17', 'tempogram_z17', 'tempogram_x18', 'tempogram_y18', 'tempogram_z18', 'tempogram_x19', 'tempogram_y19', 'tempogram_z19', 'tempogra

In [36]:
len(columns_t)

1152

In [37]:
df_t = dataframe_creator(diseased_,
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 384)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_x0,tempogram_y0,tempogram_z0,tempogram_x1,tempogram_y1,tempogram_z1,tempogram_x2,tempogram_y2,tempogram_z2,tempogram_x3,...,tempogram_z380,tempogram_x381,tempogram_y381,tempogram_z381,tempogram_x382,tempogram_y382,tempogram_z382,tempogram_x383,tempogram_y383,tempogram_z383
0,1.0,0.660973,0.547097,0.516122,0.503224,0.498063,0.492636,0.487188,0.481721,0.476237,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,1.0,0.747967,0.651484,0.641024,0.652782,0.656155,0.648741,0.641304,0.633848,0.626374,...,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.644831,0.513646,0.4676,0.433952,0.411784,0.407421,0.403039,0.398639,0.394223,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,1.0,0.679498,0.566089,0.549553,0.536972,0.511826,0.506253,0.50066,0.495047,0.489417,...,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.707751,0.59328,0.577792,0.584327,0.581103,0.574643,0.568161,0.561659,0.55514,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,1.0,0.663325,0.545111,0.525881,0.52381,0.521877,0.51614,0.510382,0.504606,0.498812,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2102,1.0,0.767477,0.690513,0.676178,0.679782,0.668459,0.660936,0.65339,0.645823,0.638237,...,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
2103,1.0,0.718678,0.619783,0.591848,0.578205,0.574079,0.567739,0.561378,0.554997,0.548597,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2104,1.0,0.685437,0.59474,0.574473,0.547533,0.540698,0.534766,0.528814,0.522841,0.516851,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [38]:
df_t.to_csv(path + 'feature_extr/four-disease-tempogram.csv')

In [39]:
df_t.isin([0]).sum(axis=0)

tempogram_x0      0
tempogram_y0      0
tempogram_z0      0
tempogram_x1      0
tempogram_y1      0
                 ..
tempogram_y382    0
tempogram_z382    0
tempogram_x383    0
tempogram_y383    0
tempogram_z383    0
Length: 1152, dtype: int64

In [40]:
df_t['tempogram_x382'][10]

7.080514153861297e-17

In [41]:
final_df = pd.concat([df, ch_df], axis =1)
final_df

,zcr_x0,zcr_y0,zcr_z0,zcr_x1,zcr_y1,zcr_z1,zcr_x2,zcr_y2,zcr_z2,zcr_x3,...,G♯_z,A_x,A_y,A_z,A♯_x,A♯_y,A♯_z,B_x,B_y,B_z
0,0.059570,0.090820,0.118164,0.130859,0.093750,0.062500,0.088867,0.152832,0.210449,0.246094,...,0.540427,0.532482,0.537383,0.639506,0.701467,0.866884,0.931928,0.955682,0.9096,0.976065
1,0.072266,0.119141,0.153320,0.156250,0.119629,0.073242,0.106445,0.151367,0.199707,0.187988,...,0.513613,0.521124,0.54084,0.604759,0.727653,0.779073,0.837661,0.978571,0.966666,1.0
2,0.067383,0.112305,0.146484,0.152344,0.111816,0.067383,0.111328,0.171875,0.217285,0.214355,...,0.49622,0.518376,0.591903,0.619725,0.712624,0.826865,0.915158,0.925569,0.962287,1.0
3,0.073242,0.103516,0.133789,0.114746,0.083496,0.053223,0.093750,0.145020,0.204590,0.207031,...,0.454881,0.554078,0.653378,0.691066,0.669254,0.914142,0.964483,0.986184,0.959988,0.938678
4,0.058105,0.092285,0.126465,0.126953,0.093750,0.059570,0.109375,0.157227,0.197754,0.202148,...,0.391695,0.43192,0.520678,0.548201,0.5654,0.682132,0.851218,0.889,1.0,0.958231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,0.032715,0.050293,0.071289,0.074707,0.058105,0.040527,0.049316,0.071777,0.097168,0.095215,...,0.909993,0.658318,0.516163,0.579417,0.4886,0.80132,0.619739,0.932921,0.88882,0.946635
2102,0.030762,0.046387,0.060059,0.061523,0.047363,0.031738,0.039551,0.059082,0.074707,0.076660,...,0.873057,0.674199,0.629744,0.656249,0.536155,0.872828,0.755182,0.942336,0.814954,0.986435
2103,0.030273,0.048340,0.066895,0.069824,0.054688,0.036621,0.041504,0.056641,0.080078,0.086914,...,0.976158,0.729124,0.652433,0.702468,0.500179,0.712968,0.642459,0.861212,0.8748,0.843556
2104,0.033203,0.045898,0.062500,0.058105,0.044922,0.032227,0.041504,0.060059,0.078613,0.080566,...,0.949705,0.74859,0.643407,0.611997,0.502888,0.8207,0.701178,0.898014,0.799819,0.896401


In [42]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcr_x0,zcr_y0,zcr_z0,zcr_x1,zcr_y1,zcr_z1,zcr_x2,zcr_y2,zcr_z2,zcr_x3,...,tempogram_z380,tempogram_x381,tempogram_y381,tempogram_z381,tempogram_x382,tempogram_y382,tempogram_z382,tempogram_x383,tempogram_y383,tempogram_z383
0,0.059570,0.090820,0.118164,0.130859,0.093750,0.062500,0.088867,0.152832,0.210449,0.246094,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,0.072266,0.119141,0.153320,0.156250,0.119629,0.073242,0.106445,0.151367,0.199707,0.187988,...,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.067383,0.112305,0.146484,0.152344,0.111816,0.067383,0.111328,0.171875,0.217285,0.214355,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.073242,0.103516,0.133789,0.114746,0.083496,0.053223,0.093750,0.145020,0.204590,0.207031,...,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.058105,0.092285,0.126465,0.126953,0.093750,0.059570,0.109375,0.157227,0.197754,0.202148,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,0.032715,0.050293,0.071289,0.074707,0.058105,0.040527,0.049316,0.071777,0.097168,0.095215,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2102,0.030762,0.046387,0.060059,0.061523,0.047363,0.031738,0.039551,0.059082,0.074707,0.076660,...,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
2103,0.030273,0.048340,0.066895,0.069824,0.054688,0.036621,0.041504,0.056641,0.080078,0.086914,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2104,0.033203,0.045898,0.062500,0.058105,0.044922,0.032227,0.041504,0.060059,0.078613,0.080566,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [43]:
final_df.to_csv(path + 'feature_extr/four-disease-final-signal_processing.csv')